
*   Madhav Sood IMT2021009
*   Vineet Priyedarshi IMT2021018
*   Shlok Agrawal IMT2021103


In [ ]:
!gdown 18Gn0ndFQooqUXzuP_aP2aEwUFZ5SKp0e
!gdown 13urXnrbu92GVmqFIkGS4mh7Mc6X0OI5-
!gdown 1FBugfSuiKYyATK60LbKtEJgCwAG5IU5Q

Downloading...
From: https://drive.google.com/uc?id=18Gn0ndFQooqUXzuP_aP2aEwUFZ5SKp0e
To: /content/train.csv
100% 12.6M/12.6M [00:00<00:00, 179MB/s]
Downloading...
From: https://drive.google.com/uc?id=13urXnrbu92GVmqFIkGS4mh7Mc6X0OI5-
To: /content/test.csv
100% 5.34M/5.34M [00:00<00:00, 155MB/s]
Downloading...
From: https://drive.google.com/uc?id=1FBugfSuiKYyATK60LbKtEJgCwAG5IU5Q
To: /content/sample_submission.csv
100% 418k/418k [00:00<00:00, 77.5MB/s]


In [ ]:
!pip install optuna
!pip install lightgbm
!pip install catboost

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.4/413.4 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 230.6/230.6 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.6/78.6 kB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.7/98.7 MB 9.2 MB/s eta 0:00:00


In [ ]:
import numpy as np
import optuna
import pandas as pd
import matplotlib.pyplot as plt
import urllib
import sklearn
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures, StandardScaler, LabelEncoder
from sklearn.metrics import accuracy_score, confusion_matrix
import seaborn as sns
import xgboost as xgb
from sklearn.ensemble import GradientBoostingClassifier
# from imblearn.over_sampling import RandomOverSampler, SMOTE
# from imblearn.under_sampling import RandomUnderSampler
from catboost import CatBoostClassifier
import lightgbm as lgb

In [ ]:
test_df = pd.read_csv("/content/test.csv")
df = pd.read_csv("/content/train.csv")


print(df.shape)
print(df['patient_id'].nunique())

(71236, 50)
54151


In [ ]:
df.head()

,enc_id,patient_id,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,...,citoglipton,insulin,glyburide-metformin,glipizide-metformin,glimepiride-pioglitazone,metformin-rosiglitazone,metformin-pioglitazone,change,diabetesMed,readmission_id
0,88346340,2488608,Caucasian,Male,[60-70),NaN,1,2,6,3,...,No,Steady,No,No,No,No,No,Ch,Yes,2
1,92001408,52133202,Caucasian,Male,[70-80),[100-125),2,6,1,7,...,No,No,No,No,No,No,No,No,Yes,1
2,169424316,40945509,Caucasian,Female,[70-80),NaN,3,2,1,7,...,No,Up,No,No,No,No,No,Ch,Yes,1
3,272987082,38850777,Caucasian,Female,[50-60),NaN,1,1,7,1,...,No,No,No,No,No,No,No,No,Yes,2
4,150600612,72738225,Caucasian,Female,[80-90),NaN,1,6,7,6,...,No,Down,No,No,No,No,No,Ch,Yes,2


In [ ]:
df.drop_duplicates(inplace=True)

In [ ]:
df.describe()

,enc_id,patient_id,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,num_lab_procedures,num_procedures,num_medications,number_outpatient,number_emergency,number_inpatient,number_diagnoses,readmission_id
count,7.123600e+04,7.123600e+04,71236.000000,71236.000000,71236.000000,71236.000000,71236.000000,71236.000000,71236.000000,71236.000000,71236.000000,71236.000000,71236.000000,71236.000000
mean,1.651772e+08,5.421577e+07,2.020116,3.721250,5.743500,4.394660,43.112612,1.342523,16.016452,0.371104,0.195800,0.635353,7.418103,1.427523
std,1.027123e+08,3.866893e+07,1.441682,5.291709,4.050373,2.979942,19.658896,1.705587,8.130801,1.269937,0.864571,1.269287,1.930480,0.684073
min,1.573800e+04,1.350000e+02,1.000000,1.000000,1.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000
25%,8.471072e+07,2.340213e+07,1.000000,1.000000,1.000000,2.000000,31.000000,0.000000,10.000000,0.000000,0.000000,0.000000,6.000000,1.000000
50%,1.523227e+08,4.520375e+07,1.000000,1.000000,7.000000,4.000000,44.000000,1.000000,15.000000,0.000000,0.000000,0.000000,8.000000,2.000000
75%,2.307948e+08,8.742009e+07,3.000000,4.000000,7.000000,6.000000,57.000000,2.000000,20.000000,0.000000,0.000000,1.000000,9.000000,2.000000
max,4.438672e+08,1.895026e+08,8.000000,28.000000,25.000000,14.000000,132.000000,6.000000,81.000000,42.000000,63.000000,19.000000,16.000000,2.000000


Check Target Variable

PreProcessing

Frequency feature


In [ ]:
train_frequency = df['patient_id'].value_counts().to_dict()
test_frequency = test_df['patient_id'].value_counts().to_dict()

# Combine train and test frequencies
combined_frequency = {key: train_frequency.get(key, 0) + test_frequency.get(key, 0) for key in set(train_frequency) | set(test_frequency)}

# Add 'frequency' column to both DataFrames
df['frequency'] = df['patient_id'].map(combined_frequency)
test_df['frequency'] = test_df['patient_id'].map(combined_frequency)


**Dropping columns of some drugs as they are very imbalanced**

In [ ]:
# Extract features and labels
X = df.drop(['enc_id', 'patient_id', 'readmission_id', 'repaglinide', 'nateglinide', 'chlorpropamide',
             'acetohexamide', 'tolbutamide', 'acarbose', 'miglitol', 'troglitazone',
             'tolazamide', 'examide', 'citoglipton', 'glyburide-metformin', 'glipizide-metformin',
             'metformin-rosiglitazone', 'metformin-pioglitazone', 'glimepiride-pioglitazone'], axis=1)
# Y = to_categorical(df['readmission_id'])
Y = df['readmission_id']
test_df = test_df.drop(['enc_id', 'patient_id', 'repaglinide', 'nateglinide', 'chlorpropamide',
             'acetohexamide', 'tolbutamide', 'acarbose', 'miglitol', 'troglitazone',
             'tolazamide', 'examide', 'citoglipton', 'glyburide-metformin', 'glipizide-metformin',
             'metformin-rosiglitazone', 'metformin-pioglitazone', 'glimepiride-pioglitazone'], axis=1)

# ['repaglinide', 'nateglinide', 'chlorpropamide',
#              'acetohexamide', 'to]lbutamide', 'acarbose', 'miglitol', 'troglitazone',
#              'tolazamide', 'examide', 'citoglipton', 'glyburide-metformin', 'glipizide-metformin',
#              'metformin-rosiglitazone', 'metformin-pioglitazone', 'glimepiride-pioglitazone']


numerical_columns = X.select_dtypes(include=['int64', 'float64']).columns
categorical_columns = X.select_dtypes(include=['object']).columns

In [ ]:
column_name_to_check = 'readmission_id'

# Check if the column is present in test_df_encoded
if column_name_to_check in X.columns:
    print(f"The column '{column_name_to_check}' is present in test_df_encoded.")
else:
    print(f"The column '{column_name_to_check}' is NOT present in test_df_encoded.")


The column 'readmission_id' is NOT present in test_df_encoded.


In [ ]:
print(len(X.columns))
print(len(test_df.columns))

32
32


In [ ]:
print(df.shape)

(71236, 51)


In [ ]:
X.head()

,race,gender,age,weight,admission_type_id,discharge_disposition_id,admission_source_id,time_in_hospital,payer_code,medical_specialty,...,metformin,glimepiride,glipizide,glyburide,pioglitazone,rosiglitazone,insulin,change,diabetesMed,frequency
0,Caucasian,Male,[60-70),NaN,1,2,6,3,HM,Family/GeneralPractice,...,No,No,No,Steady,No,No,Steady,Ch,Yes,1
1,Caucasian,Male,[70-80),[100-125),2,6,1,7,MC,NaN,...,Steady,No,No,No,No,No,No,No,Yes,2
2,Caucasian,Female,[70-80),NaN,3,2,1,7,SP,NaN,...,No,No,No,No,No,No,Up,Ch,Yes,8
3,Caucasian,Female,[50-60),NaN,1,1,7,1,MD,NaN,...,Steady,No,No,No,No,No,No,No,Yes,1
4,Caucasian,Female,[80-90),NaN,1,6,7,6,MC,NaN,...,No,No,No,No,No,No,Down,Ch,Yes,1


In [ ]:


print("Categorical Columns:", len(categorical_columns))
print("Numerical Columns:", len(numerical_columns))


Categorical Columns: 20
Numerical Columns: 12


DUMMY ENCODING


In [ ]:
print(numerical_columns)

Index(['admission_type_id', 'discharge_disposition_id', 'admission_source_id',
       'time_in_hospital', 'num_lab_procedures', 'num_procedures',
       'num_medications', 'number_outpatient', 'number_emergency',
       'number_inpatient', 'number_diagnoses', 'frequency'],
      dtype='object')


One hot encoding

In [ ]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

pre processing of both test and train data

In [ ]:
# Impute missing values for numerical columns
numerical_imputer = SimpleImputer(strategy='most_frequent')
X[numerical_columns] = numerical_imputer.fit_transform(X[numerical_columns])
test_df[numerical_columns] = numerical_imputer.transform(test_df[numerical_columns])

# Impute missing values for categorical columns
categorical_imputer = SimpleImputer(strategy='most_frequent')
X[categorical_columns] = categorical_imputer.fit_transform(X[categorical_columns])
test_df[categorical_columns] = categorical_imputer.transform(test_df[categorical_columns])

# One-hot encode categorical columns
categorical_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
X_encoded = categorical_encoder.fit_transform(X[categorical_columns])
test_data_processed = categorical_encoder.transform(test_df[categorical_columns])

# Combine numerical and encoded categorical features
X_final = pd.concat([X[numerical_columns], pd.DataFrame(X_encoded, columns=categorical_encoder.get_feature_names_out(categorical_columns))], axis=1)
test_df_encoded = pd.concat([test_df[numerical_columns], pd.DataFrame(test_data_processed, columns=categorical_encoder.get_feature_names_out(categorical_columns))], axis=1)

# Split the training data for training and validation
X_train, X_test, Y_train, Y_test = train_test_split(X_final, Y, test_size=0.2, random_state=42)


/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_encoders.py:868: FutureWarning: `sparse` was renamed to `sparse_output` in version 1.2 and will be removed in 1.4. `sparse_output` is ignored unless you leave `sparse` to its default value.
  warnings.warn(


In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, f1_score

**SVM**

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [ ]:
# SVM with Polynomial Kernel
svm_poly = SVC(kernel='poly', degree=3, random_state=42)
svm_poly.fit(X_train, Y_train)


SVC(kernel='poly', random_state=42)

predict on the train set

In [ ]:
from sklearn.metrics import accuracy_score, classification_report, f1_score

In [ ]:
# # Predict on the test set
y_pred = svm_poly.predict(X_test)

In [ ]:

# # Evaluate the model
accuracy = accuracy_score(Y_test, y_pred)
print("Accuracy:", accuracy)
f1_macro = f1_score(Y_test, y_pred, average='macro')
print(f1_macro)

# # Print the classification report
print("Classification Report:")
print(classification_report(Y_test, y_pred))

Accuracy: 0.6348961257720381
0.41336029311356626
Classification Report:
              precision    recall  f1-score   support

           0       0.00      0.00      0.00      1710
           1       0.61      0.39      0.48      4989
           2       0.64      0.94      0.76      7549

    accuracy                           0.63     14248
   macro avg       0.42      0.44      0.41     14248
weighted avg       0.55      0.63      0.57     14248



/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
# SVM with RBF Kernel
svm_rbf = SVC(kernel='rbf', random_state=42)
svm_rbf.fit(X_train, Y_train)
#y_pred_rbf = svm_rbf.predict(X_val)
#accuracy_rbf = accuracy_score(y_val, y_pred_rbf)
#print(f'RBF Kernel Accuracy: {accuracy_rbf}')
# Predict on the test set
y_pred = svm_rbf.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(Y_test, y_pred)
print("Accuracy:", accuracy)

f1_macro = f1_score(Y_test, y_pred, average='macro')
print("F1 Macro:", f1_macro)

# Print the classification report
print("Classification Report:")
print(classification_report(Y_test, y_pred))

Prediction for svm

In [ ]:
len(X_final.columns)

2282

In [ ]:
len(test_df_encoded.columns)

2282

In [ ]:

y_pred_data = svm_poly.predict(test_df_encoded)
subm = pd.read_csv("/content/sample_submission.csv")
subm["readmission_id"] = y_pred_data
subm.to_csv('submission.csv', index=False)
